# Update Ollama
# curl -fsSL https://ollama.com/install.sh | sh
# !/set parameter num_ctx 4096

## LLM Chain

In [30]:
from langchain_community.llms import Ollama

llm = Ollama(model="mistral", num_ctx=4096)


In [5]:
llm.invoke("how can you make a Ratatouille with only meat?")


"\nYou can make a Ratatouille with only meat by substituting the vegetables in the recipe. Here's a simple recipe that you can try:\n\nIngredients:\n\n* 1 pound of ground beef or turkey\n* 1 large onion, chopped\n* 3 cloves of garlic, minced\n* 2 red bell peppers, diced\n* 2 zucchinis, sliced\n* 1 eggplant, sliced\n* 1 can of diced tomatoes\n* 1 teaspoon dried thyme\n* 1 teaspoon dried basil\n* Salt and pepper to taste\n\nInstructions:\n\n1. In a large skillet, cook the ground beef or turkey over medium heat until browned. Drain off any excess fat.\n2. Add the chopped onion and minced garlic to the skillet and cook until softened, about 5 minutes.\n3. Add the diced red bell peppers, sliced zucchinis, and sliced eggplant to the skillet and cook for another 5-7 minutes, stirring occasionally, until the vegetables are tender.\n4. Pour in the canned tomatoes, dried thyme, dried basil, salt, and pepper into the skillet. Stir well to combine.\n5. Reduce the heat to low and let the Ratatouill

In [6]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

chain = prompt | llm 

chain.invoke({"input": "how can langsmith help with testing?"})


'\nAs a world-class technical documentation writer, I can explain how Langsmith can help with testing in the context of technical writing. Langsmith is an AI language model that helps businesses and individuals create high-quality written content. In the context of testing, Langsmith can be used to help document the process of testing a product or system.\n\nTo use Langsmith for testing documentation, you can start by outlining the goals and objectives of your testing process. This will help you determine the scope of your testing and the types of tests you need to run. You can then use Langsmith to generate test cases based on these goals and objectives. Langsmith can also help you create test plans and procedures, as well as provide guidance on how to interpret test results and identify issues.\n\nAdditionally, Langsmith can be used to document the steps involved in running tests, including any necessary set-up or configuration requirements. This documentation can be useful for both 

In [7]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

chain = prompt | llm | output_parser

chain.invoke({"input": "how can langsmith help with testing?"})



"\nAs a world class technical documentation writer, I understand the importance of testing in software development. Langsmith can provide several ways to help with testing:\n\n1. Test case creation: Langsmith's natural language processing (NLP) capabilities can help generate test cases based on requirements and specifications. This can save time and effort for developers who have to write test cases manually.\n2. Bug detection: Langsmith can be used to identify potential bugs or errors in code by analyzing the syntax, structure, and semantics of the language used. This can help catch issues early in the development cycle, reducing the time and resources required for fixing them.\n3. Code quality assessment: Langsmith's NLP capabilities can also be used to assess the quality of code. By analyzing the codebase, Langsmith can identify best practices, coding standards, and potential security vulnerabilities, helping developers improve the overall code quality.\n4. Regression testing: Langs

# Retrieval Chain

In [8]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = loader.load()

In [9]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings()

In [10]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [11]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [12]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'Langsmith can help with testing by allowing you to visualize test results.'

In [13]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [14]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...

LangSmith can help with testing LLM applications by providing a platform for creating datasets, running tests, and comparing test results. LangSmith allows developers to create datasets, which are collections of inputs and reference outputs, and use these to run tests on their LLM applications. These test cases can be uploaded in bulk, created on the fly, or exported from application traces. LangSmith also makes it easy to run custom evaluations (both LLM and heuristic based) to score test results. The comparison view enables users to see whether or not they've regressed with respect to their initial test cases when making changes in the prompt, retrieval strategy, or model choice. Additionally, LangSmith provides a playground environment for rapid iteration and experimentation, allowing developers to quickly test out different prompts and models. Every playground run is logged in the system and can be used to create test cases or compare with other runs.


## Conversation Retrieval Chain

In [15]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [16]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content='Skip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubProxyUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping\u200bPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly understand how the model is performing — and debug where it is failing — is incredibly important for this phase.Debugging\u200bWhen developing new LLM applications,

In [17]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [18]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content='LangSmith User Guide | 🦜️🛠️ LangSmith', metadata={'source': 'https://docs.smith.langchain.com/user_guide', 'title': 'LangSmith User Guide | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.', 'language': 'en'}),
  Document(page_content="This allows you to quickly test out different prompts and models. You can open the playground from any prompt or model run in your trace.\nEvery playground run is logged in the system and can be used to create test cases

## Output Parsers

In [19]:
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator

In [20]:
model = llm

In [29]:
from openai import OpenAI
from pydantic import BaseModel, Field
from typing import List

import instructor


class Character(BaseModel):
    name: str
    age: int
    fact: List[str] = Field(..., description="A long list of facts about the character")


# enables `response_model` in create call
client = instructor.patch(
    OpenAI(
        base_url="http://localhost:11434/v1",
        api_key="ollama",  # required, but unused
    ),
    mode=instructor.Mode.JSON,
)

resp = client.chat.completions.create(
    model="llama2",
    messages=[
        {
            "role": "user",
            "content": "Tell me everything you know about Harry Potter",
        }
    ],
    response_model=Character,
)
print(resp.model_dump_json(indent=2))
print("""
{
  "name": "Harry James Potter",
  "age": 37,
  "fact": [
    "He is the chosen one.",
    "He has a lightning-shaped scar on his forehead.",
    "He is the son of James and Lily Potter.",
    "He attended Hogwarts School of Witchcraft and Wizardry.",
    "He is a skilled wizard and sorcerer.",
    "He fought against Lord Voldemort and his followers.",
    "He has a pet owl named Snowy."
  ]
}
""")

{
  "name": "Harry Potter",
  "age": 37,
  "fact": [
    "He is a wizard and the chosen one.",
    "He has a lightning-shaped scar on his forehead.",
    "He attended Hogwarts School of Witchcraft and Wizardry.",
    "He is the son of James and Lily Potter.",
    "He has a pet owl named Snowy.",
    "He is known for his bravery and willingness to risk his own life to save others."
  ]
}

{
  "name": "Harry James Potter",
  "age": 37,
  "fact": [
    "He is the chosen one.",
    "He has a lightning-shaped scar on his forehead.",
    "He is the son of James and Lily Potter.",
    "He attended Hogwarts School of Witchcraft and Wizardry.",
    "He is a skilled wizard and sorcerer.",
    "He fought against Lord Voldemort and his followers.",
    "He has a pet owl named Snowy."
  ]
}

